In [1]:
import geemap
import ee
geemap.ee_initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=eWIaGIcwy1PhSn9qdap951igix-_Lhr2IEdfIm-XV-M&tc=bxlBa6dz4c5BoPtx1YwWrwRnXQbM-MqMfNmB-PtVdhA&cc=2gvf7q4sClrEgYKkgqP5Ys7yIa9cxBEReS7t-IRP674

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AeaYSHBezYFTa_-JFKTua2A75OE0lFtCozq1lqwPxzv-I1lSmhXjj3XEh58

Successfully saved authorization token.


**Question 1**

In [10]:
Map=geemap.Map()
#url='https://imgur.com/spILFEi' #The internet link doesn't work
url='NOAA_logo.png' #I put the image localy
Map.add_image(url, position='bottomleft')
dataset = ee.ImageCollection('NOAA/GFS0P25') \
                  .filter(ee.Filter.date('2018-03-01', '2018-03-02'))
temperatureAboveGround = dataset.select('temperature_2m_above_ground')
vis_params = {
  'min': -40.0,
  'max': 40.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
Map.setCenter(71.72, 52.48, 3.0)
Map.add_text('Made by Javier Martinez', fontsize=20)
Map.addLayer(temperatureAboveGround, vis_params, 'Temperature Above Ground')
Map.add_colorbar(vis_params, label="Temperature above ground (ºC)", layer_name="SRTM DEM")
Map

Map(center=[52.48, 71.72], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

**Question 2**

In [12]:
image = (
    ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
)
vis_params = [
    {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 120},
    {'bands': ['B4', 'B5', 'B3'], 'min': 0, 'max': 120},
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 120},
    {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 120},
]
labels = [
    'Natural Color (B3/B2/B1)',
    'Land/Water (B4/B5/B3)',
    'Color Infrared (B4/B3/B2)',
    'Vegetation (B5/B4/B3)',
]
geemap.linked_maps(
    rows=2,
    cols=2,
    height="300px",
    center=[38.4151, 21.2712],
    zoom=12,
    ee_objects=[image],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

**Question 3**

In [14]:
Map = geemap.Map(center=[40, -100], zoom=4)
collection = ee.ImageCollection('USDA/NASS/CDL').select('cropland')
vis_params = {'bands': ['cropland']}
years = collection.aggregate_array('system:index').getInfo()
Map.ts_inspector(
    left_ts=collection,
    right_ts=collection,
    left_names=years,
    right_names=years,
    left_vis=vis_params,
    right_vis=vis_params,
    width='80px',
)
Map.add_text('Made by Javier', fontsize=20)
url2 = 'USDA_LOGO.png'
Map.add_image(url2, position='bottomleft')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=…

**Question 4**

In [5]:
from geemap.datasets import DATA
Map=geemap.Map()
region2 =ee.FeatureCollection(DATA.TIGER_2018_Counties).filter(ee.Filter.eq('GEOID', '47093'))
vis = {
    'color': '00000000',
    'lineType': 'solid',
}
Map.centerObject(region2, zoom=13)
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2023-06-01', '2023-07-30')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .filterBounds(region2)
)
sentinel=dataset.mean()
visualization = {
    'min': 0.0,
    'max': 7000,
    'bands': ['B8', 'B3', 'B4'],
}
clip=sentinel.clipToCollection(region2)
Map.add_layer(clip, visualization, 'RGB')
Map.add_text('Made by Javier Martinez', fontsize=20)
Map.add_time_slider(dataset, visualization)
Map

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

**Question 5**

In [15]:
from ipyleaflet import Map, basemaps
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
collection = ee.ImageCollection("LANDSAT/LE7_TOA_5YEAR").first()
bands = collection.select(['B3', 'B2', 'B1'])
vis = {
    'min':0.0,
    'max': 100.0,
}
landsat=bands.clipToCollection(states)
collection2= ee.ImageCollection("ESA/WorldCover/v100").first()
vis2 = { bands: ['Map']}
esa = collection2.clipToCollection(states)
left_layer = geemap.ee_tile_layer(landsat, vis, 'Landsat')
right_layer = geemap.ee_tile_layer(esa, vis2, 'ESA Worldcover')
Map.split_map(left_layer, right_layer)
Map.add_legend(builtin_legend='ESA_WorldCover', title= 'ESA Land Cover Type')
Map.add_text('Made by Javier Martinez', fontsize=15)
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…